# Main file for thesis project

Ie the top level script which runs the whole `pic -> recognise -> aim gimbal -> repeat` show

In [ ]:
import time
import numpy as np

# import gimbal_control as gc

## A Basic Loop

In [ ]:
from extended_kalman_filter import ExtendedKalmanFilter

EKF_yaw = ExtendedKalmanFilter(Ts=0.05, Q=0.05, R=0.1, a=0.99)  # need to tweak Q and R
EKF_pitch = ExtendedKalmanFilter(Ts=0.05, Q=0.05, R=0.1, a=0.99)  # need to tweak Q and R

In [ ]:
from image_classifier import ImageClassifier

IC = ImageClassifier(
    graph_filename='../Models/MobileNet_SSD_caffe/graph',
    label_filename='../Models/MobileNet_SSD_caffe/categories.txt',
    class_of_interest='person',
    camera_resolution=(1640,922),
    debug=False)

In [ ]:
phi_yaw_arr = []
EKF_yaw_arr = []

phi_pitch_arr = []
EKF_pitch_arr = []

phi_yaw = 0
phi_pitch = 0

time_arr = []
t_start = time.time()

# constants
total_run_time = 30 # seconds
t_loop = 0.05       # seconds = 50 ms

while True:
    t = time.time()
    print('+', end='')
    
    EKF_yaw.predict()
    EKF_pitch.predict()
    
    # if there is a new result waiting...
    if not IC.dict_queue.empty():
        print('.',end='')
        bb, bb_angles = IC.get_result()

        # and there was an actual object detected...
        if bb_angles != -1:
            (phi_x1, phi_y1), (phi_x2, phi_y2) = bb_angles
            phi_yaw = (phi_x1 + phi_x2)/2
            phi_pitch = (phi_y1 + phi_y2)/2

            EKF_yaw.update(phi_yaw)
            EKF_pitch.update(phi_pitch)

    phi_yaw_arr.append(phi_yaw)
    EKF_yaw_arr.append(EKF_yaw.x.item(0))
    phi_pitch_arr.append(phi_pitch)
    EKF_pitch_arr.append(EKF_pitch.x.item(0))
    
    time_arr.append(time.time() - t_start)
    
    if time.time() > t_start + total_run_time:
        break
    
    if time.time() - t < 0.001:
        continue
    else:
        time.sleep(t_loop - (time.time() - t))  # aim for a loop time of 50ms

IC.close()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# df = pd.DataFrame(
#     columns=['phi_yaw_arr', 'phi_pitch_arr', 'time_arr'],
#     data=np.array([phi_yaw_arr, phi_pitch_arr, time_arr]).T)
# df.to_csv('logged_data_2.csv')

In [ ]:
# df_loaded = pd.read_csv('logged_data_1.csv', index_col=0)
# print(df_loaded.columns)

# phi_yaw_arr = df_loaded.iloc[:,0].values
# phi_pitch_arr = df_loaded.iloc[:,1].values
# time_arr = df_loaded.iloc[:,2].values

In [ ]:
plt.plot(time_arr, EKF_yaw_arr, label='EKF estimate of yaw [deg]')
plt.plot(time_arr, phi_yaw_arr, label='Raw NN estimate of yaw [deg]')
plt.legend(); plt.grid(); fig = plt.gcf(); fig.set_size_inches(18.5, 5, forward=True); plt.show()

In [ ]:
plt.plot(time_arr, EKF_pitch_arr, label='EKF estimate of pitch [deg]')
plt.plot(time_arr, phi_pitch_arr, label='Raw NN estimate of pitch [deg]')
plt.legend(); plt.grid(); fig = plt.gcf(); fig.set_size_inches(18.5, 5, forward=True); plt.show()

In [ ]:
loop_times_ms = [(t-t_)*1e3 for t_,t in zip(time_arr[0:-1], time_arr[1:])]
plt.stem(time_arr[:-1], loop_times_ms, label='loop times [ms]')
plt.legend(); plt.grid(); fig = plt.gcf(); fig.set_size_inches(18.5, 5, forward=True); plt.show()

In [ ]:
# def plot_util(_plt):
#     _plt.legend()
#     _plt.grid()
#     fig = _plt.gcf();
#     fig.set_size_inches(18.5, 5, forward=True)
#     _plt.show()